In [1]:
import pickle
import os, sys
sys.path.append(os.path.join(os.getcwd(), '..')) # this is the ml_dadi dir
import ml_models
from ml_models import rfr_train, mlpr_train


In [2]:
# Load training datasets
names = ['full_no_log', 'full_log', 'exclude_log']
train_data = {}
for name in names:
    list_train_dict = pickle.load(open(f'data/train_data_{name}', 'rb'))
    train_data[name] = list_train_dict


In [ ]:
# # Train RFR
# for name in names:
#     list_train_dict = train_data[name]
#     list_rfr = [rfr_train(train_dict) for train_dict in list_train_dict]
#     pickle.dump(list_rfr, open(f'data/list_rfr_{name}', 'wb'), 2)


In [ ]:
# # Train MLPR with lbfgs solver
# # Need large max_iter and take longer to run but perform the best for two_epoch
# for name in names:
#     print(name)
#     list_train_dict = train_data[name]
#     list_mlpr_lbfgs = [mlpr_train(train_dict, hidden_layer_sizes=(25,25,25,25),
#                                             solver='lbfgs', max_iter=5000)
#                             for train_dict in list_train_dict]
#     pickle.dump(list_mlpr_lbfgs, open(f'data/list_mlpr_lbfgs_{name}', 'wb'), 2)

In [14]:
# Train MLPR with lbfgs solver for exclude_log and theta=1
train_dict = train_data['exclude_log'][0]

mlpr_lbfgs = mlpr_train(train_dict, hidden_layer_sizes=(25,25,25,25),
                                solver='lbfgs', max_iter=1000, alpha=1)
pickle.dump(mlpr_lbfgs, open(f'data/mlpr_lbfgs_exclude_log_new', 'wb'), 2)